In [2]:
import rasterio
from ipyleaflet import Map, ImageOverlay
from ipywidgets import  VBox, FloatRangeSlider
import numpy as np
import PIL.Image
from io import BytesIO
import base64

# Read the TIFF file
tiff_file = r"C:\development\coastseg-planet\CoastSeg-Planet\Santa Cruz Boardwalk_topobathy.tif"
with rasterio.open(tiff_file) as src:
    mask_data = src.read(1)  # Read the first band
    bounds = src.bounds
    transform = src.transform

# Function to create a base64 PNG image from mask data with transparency
def create_masked_image(mask_data, alpha=128):
    # Normalize the mask data to 0-255
    mask_data = (mask_data / mask_data.max() * 255).astype(np.uint8)
    # Create an RGBA image with the red channel filled and alpha channel set by the mask
    rgba_image = np.zeros((mask_data.shape[0], mask_data.shape[1], 4), dtype=np.uint8)
    rgba_image[..., 0] = mask_data  # Red channel
    # Set alpha to the specified value for the masked regions, 0 for the unmasked regions
    rgba_image[..., 3] = np.where(mask_data > 0, alpha, 0)  # Alpha channel (transparency)
    img = PIL.Image.fromarray(rgba_image)
    with BytesIO() as buffer:
        img.save(buffer, format="png")
        return base64.b64encode(buffer.getvalue()).decode('utf-8')

# Create the initial masked image overlay
mask_data = np.flipud(mask_data)  # Flip the mask data vertically
masked_image = create_masked_image(mask_data)
image_overlay = ImageOverlay(
    url="data:image/png;base64," + masked_image,
    bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]]
)

# Create the map
m = Map(center=[(bounds.top + bounds.bottom) / 2, (bounds.left + bounds.right) / 2], zoom=15)
m.add_layer(image_overlay)

# Function to update the mask (simulated here for demonstration purposes)
def update_mask(change):
    global image_overlay
    if not isinstance(change, tuple):
        mask = mask_data < change
        masked_image = create_masked_image(mask)
        image_overlay.url = "data:image/png;base64," + masked_image
    else:
        lower, upper = change
        mask = np.logical_and(mask_data > lower, mask_data < upper)
        masked_image = create_masked_image(mask)
        image_overlay.url = "data:image/png;base64," + masked_image



# Create a bounded slider with two handles
threshold_slider = FloatRangeSlider(
    description='Elevation (m)',
    value=[default_value[0], default_value[1]],
    min=mask_data.min(),
    max=mask_data.max(),
    step=0.1,
)

# Link the slider to the update function
threshold_slider.observe(lambda change: update_mask(change['new']), names='value')

# Display the map and the slider
display(VBox([m, threshold_slider]))

default_value = (-10, 10)
# Initial display
update_mask(default_value)
